# Loop 29 Strategic Analysis

## Current Status
- Best CV: 70.624381
- Best LB: 70.626088 (exp_024)
- Target: 68.919154
- Gap: 1.705 (2.41%)

## Key Question
After 30 experiments with ZERO improvement in the last 5, what fundamentally different approach can close the 1.7 point gap?

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
import matplotlib.pyplot as plt

TREE_TEMPLATE = [
    (0.0, 0.8), (0.125, 0.5), (0.0625, 0.5), (0.2, 0.25), (0.1, 0.25),
    (0.35, 0.0), (0.075, 0.0), (0.075, -0.2), (-0.075, -0.2), (-0.075, 0.0),
    (-0.35, 0.0), (-0.1, 0.25), (-0.2, 0.25), (-0.0625, 0.5), (-0.125, 0.5)
]

def parse_s_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return float(val[1:])
    return float(val)

def create_tree_polygon(x, y, angle):
    tree = Polygon(TREE_TEMPLATE)
    tree = rotate(tree, angle, origin=(0, 0), use_radians=False)
    tree = translate(tree, x, y)
    return tree

def get_n_score(df, n):
    group = df[df['n'] == n]
    trees = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in group.iterrows()]
    all_x, all_y = [], []
    for tree in trees:
        minx, miny, maxx, maxy = tree.bounds
        all_x.extend([minx, maxx])
        all_y.extend([miny, maxy])
    side = max(max(all_x) - min(all_x), max(all_y) - min(all_y))
    return (side ** 2) / n, side

# Load current submission
df = pd.read_csv('/home/submission/submission.csv')
df['x'] = df['x'].apply(parse_s_value)
df['y'] = df['y'].apply(parse_s_value)
df['deg'] = df['deg'].apply(parse_s_value)
df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))

print(f'Loaded {len(df)} rows')

In [ ]:
# Calculate scores for each N
scores = []
for n in range(1, 201):
    score, side = get_n_score(df, n)
    scores.append({'n': n, 'score': score, 'side': side})

scores_df = pd.DataFrame(scores)
total_score = scores_df['score'].sum()
print(f'Total score: {total_score:.6f}')
print(f'Target: 68.919154')
print(f'Gap: {total_score - 68.919154:.6f} ({(total_score - 68.919154)/68.919154*100:.2f}%)')

In [ ]:
# STRATEGIC CONCLUSION
tree_area = Polygon(TREE_TEMPLATE).area
scores_df['theoretical_min_side'] = np.sqrt(scores_df['n'] * tree_area)
scores_df['efficiency'] = (scores_df['theoretical_min_side'] / scores_df['side']) ** 2
best_efficiency = scores_df['efficiency'].max()

print('=== STRATEGIC CONCLUSION ===')
print(f'''
Current score: {total_score:.6f}
Target: 68.919154
Gap: {total_score - 68.919154:.6f} (2.41%)

KEY FINDINGS:
1. Best observed efficiency: {best_efficiency*100:.1f}% (for large N)
2. Small N (1-20) efficiency: ~55-65%
3. Large N (100+) efficiency: ~70-73%

THE GAP ANALYSIS:
- To reach 68.919, we need 2.41% improvement
- This requires either:
  a) Uniform 2.41% improvement across all N (unlikely with local search)
  b) Significant improvement on specific N values
  c) A fundamentally different packing approach

WHAT WE'VE TRIED (ALL FAILED):
- SA optimization (converges to same local optimum)
- bbox3 optimization (produces overlaps)
- Grid-based solutions (25% worse)
- Ensemble from multiple sources (exhausted)
- Asymmetric perturbations (no improvement)
- Beam search (all worse)

WHAT HASN'T BEEN TRIED:
1. SparroWASM professional nesting solver
2. MIP/CP exact solver for small N
3. Fundamentally different initial configurations
4. Per-N specialized algorithms

RECOMMENDATION:
The baseline is at a STRONG LOCAL OPTIMUM. All local search methods converge to it.
We need to either:
1. Find solutions from a DIFFERENT BASIN (SparroWASM, MIP)
2. Accept that 70.62 may be close to the practical optimum for public methods
''')

print(f'\nNOTE: Our score (70.624) beats the public LB leader (71.191) by 0.567 points!')
print('This suggests we have one of the best public solutions available.')